In [715]:
import pandas as pd
import glob
import numpy as numpy
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm
import time
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
import seaborn as sns
import holidays
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, BatchNormalization, TimeDistributed, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, Callback
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.constraints import max_norm
from multiprocessing import Process, Pool

import os
import multiprocessing as mp
import warnings
from filterpy.kalman import KalmanFilter
from filterpy.common import Q_discrete_white_noise

warnings.filterwarnings('ignore')

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [804]:
EARLIEST_DATE = '2012-01-01'
N_COMPONENTS=100

# REad in data

In [573]:

def read_in_data(check_recent_date=True, recent_date_check=datetime.datetime.now().date()):
    dict_of_stocks_and_dfs = {}
    for file_ in glob.glob('../data/updated_historical_stock_and_etf_data/*.csv'):
        stock_name = file_.rsplit("/")[-1].split('_')[0].lower() 
        print(f"Reading in {stock_name}")
        df_  = pd.read_csv(f"{file_}")
        # ensure we have the most recent data
        most_recent_date = pd.to_datetime(df_.date.max())
        oldest_date = pd.to_datetime(df_.date.min())
        
        oldest_date_bool = oldest_date < datetime.datetime(2006,1,1).date()
        recent_date_bool = most_recent_date == recent_date_check
        
        if oldest_date_bool and recent_date_bool:
            dict_of_stocks_and_dfs[stock_name] = df_.sort_values('date')
        elif oldest_date_bool and not check_recent_date:
            dict_of_stocks_and_dfs[stock_name] = df_.sort_values('date')            
        else:
            print(f"Stock {stock_name} most recent date is {most_recent_date} oldest date is {oldest_date}. Skipping it")
    return dict_of_stocks_and_dfs

In [574]:
dict_of_stocks_and_dfs = read_in_data(recent_date_check=datetime.datetime(2020,5,15).date())

Reading in wal
Reading in fsbw
Stock fsbw most recent date is 2020-05-15 00:00:00 oldest date is 2012-07-10 00:00:00. Skipping it
Reading in pfbc
Stock pfbc most recent date is 2020-05-15 00:00:00 oldest date is 2011-07-21 00:00:00. Skipping it
Reading in hasi
Stock hasi most recent date is 2020-05-15 00:00:00 oldest date is 2013-04-18 00:00:00. Skipping it
Reading in mtb
Reading in rbb
Stock rbb most recent date is 2020-05-15 00:00:00 oldest date is 2017-07-26 00:00:00. Skipping it
Reading in jpm
Reading in umpq
Reading in cvbf
Reading in fitb
Reading in irm
Reading in wfc
Reading in cwbc
Reading in bsrr
Reading in key
Reading in ewbc
Reading in dlr
Reading in ubfo
Reading in cvcy
Reading in hope
Stock hope most recent date is 2020-05-15 00:00:00 oldest date is 2011-12-05 00:00:00. Skipping it
Reading in pacw
Reading in ntrs
Reading in agnc
Stock agnc most recent date is 2020-05-15 00:00:00 oldest date is 2008-05-14 00:00:00. Skipping it
Reading in cma
Reading in nrz
Stock nrz most re

In [583]:
(len(dict_of_stocks_and_dfs.keys())-1) * len(dict_of_stocks_and_dfs.keys())

1482

# Create correlation features

In [599]:
def build_correlation_dfs(dict_of_stocks_and_dfs, n_day_rolling_features_list=[ 5, 7, 10, 30, 180, 365], verbose=False):
    """
    Create correlation + variance based  upon daily closing stock prices for given date ranges
    
    also include daily volume
    
    We are trying to  predict 7 day correaltion
    """

    stock_features_dict = defaultdict(pd.DataFrame)
    start_time = time.time()
    
    start = time.time()
    n_stocks = len(dict_of_stocks_and_dfs.keys())
    final_feature_df = create_date_dummy_df()
    pairs_of_stocks = []
    
    for idx, first_stock_name in enumerate(dict_of_stocks_and_dfs.keys()):
        print('')
        print(f"Finished {idx/n_stocks} pct of stocks")
        print('')
        for second_idx, second_stock_name in enumerate(dict_of_stocks_and_dfs.keys()):
            stock_pair = f"{first_stock_name}_{second_stock_name}"
            reverse_pair = f"{second_stock_name}_{first_stock_name}"
            
            if (first_stock_name == second_stock_name) or (stock_pair in pairs_of_stocks)  or (reverse_pair in pairs_of_stocks): # pnr -> ual same as ual -> pnr
                continue
            else:
                pairs_of_stocks.append(stock_pair)
            if verbose:
                print('-------')
                print(f"{first_stock_name} & {second_stock_name}")
                print('-------')
            
            # here the date is not the index, yet
            first_stock_df = dict_of_stocks_and_dfs[f"{first_stock_name}"].loc[ 
                dict_of_stocks_and_dfs[f"{first_stock_name}"].date.isin(dict_of_stocks_and_dfs[f"{second_stock_name}"].date), :]

            #  filter second df by the dates in first

            # here the date is not the index, yet
            second_stock_df = dict_of_stocks_and_dfs[f"{second_stock_name}"].loc[ 
                dict_of_stocks_and_dfs[f"{second_stock_name}"].date.isin(first_stock_df.date), :]
            
            # set the date as an index and sort by date
            first_stock_df = first_stock_df.sort_values('date')
            second_stock_df = second_stock_df.sort_values('date')

            first_stock_df = first_stock_df.set_index('date')
            second_stock_df = second_stock_df.set_index('date')
            
            all_features_df = pd.DataFrame()
            for rolling_idx, rolling_day in enumerate(n_day_rolling_features_list):
                if verbose:
                    print(f"Rolling calculations for {rolling_day}")
                features_df = create_correlation_and_variance_features(
                    first_stock_df, second_stock_df, rolling_day, final_feature_df, 
                    first_stock_name=first_stock_name, second_stock_name=second_stock_name)
                   
                current_feature_cols = set(features_df.columns)
                final_feature_cols = set(final_feature_df.columns)

                
                if (f"{first_stock_name}_volume" not in final_feature_df.columns) and (rolling_idx == 0):
                    features_df[f"{first_stock_name}_volume"] = list(first_stock_df.volume)
                
                if (f"{second_stock_name}_volume" not in final_feature_df.columns) and (rolling_idx == 0):
                    features_df[f"{second_stock_name}_volume"] = list(second_stock_df.volume)
                    
                if rolling_idx == 0: 
                    all_features_df = features_df
                else:
                    all_features_df = all_features_df.join(features_df, on='date', lsuffix='_left')
            

                    
            all_features_df.index = pd.to_datetime(all_features_df.index)
            final_feature_df = final_feature_df.join(all_features_df, on='date')

            if verbose:
                end = time.time()
                print(f"Building all features took {(end-start)/60} minutes")
                start = time.time()

    end_time = time.time()
    print(f"Total time {(end_time-start_time) / 60} minutes for {len(pairs_of_stocks)} pairs")
    final_feature_df = add_time_feature(final_feature_df)
    return final_feature_df, pairs_of_stocks
            
        

# Note: will eventuall need to add in 0s for stocks withour correlation data with other stocks due to date range

In [600]:
def create_date_dummy_df(start_date=datetime.datetime(1980,1,1), n_years=50):
    
    #  create dummy df with dates to join against
    list_of_dates  = []
    n_days = 365*n_years
    start_date = start_date

    for i in range(n_days):
        list_of_dates.append(start_date + datetime.timedelta(i))
    df_ = pd.DataFrame(list_of_dates, columns=['date'])
    
    df_.date_ =  pd.to_datetime(df_.date)
    return df_.set_index('date')
    

In [601]:
def add_time_feature(final_stock_df):
    
    days = [i.day for i in final_stock_df.index]
    months = [i.month for i in final_stock_df.index]
    quarters = [i.quarter for i in final_stock_df.index]
    years = [i.year for i in final_stock_df.index]
    
    us_holidays = holidays.UnitedStates()
    
    h_ = np.array([i in us_holidays for i in final_stock_df.index]).astype(int)


    final_stock_df['day'] = days
    final_stock_df['month'] = months
    final_stock_df['quarter'] = quarters
    final_stock_df['year'] = years
#     final_stock_df['is_holiday'] = h_
    
    return final_stock_df

In [602]:
def create_correlation_and_variance_features(first_stock_df, second_stock_df, n_days_stride, final_stock_df, 
                                             first_stock_name=None, second_stock_name=None, verbose=False):
    """
    n_days_stride: the  number of rolling days to calculate correlation for
    """
    n_rows = len(first_stock_df)

    previous_row = 0

    features_per_time_period = defaultdict(list)
    if verbose:
        print(f"Creating correlations + variance on close for {n_days_stride} days")
    
    rolling_close_df = pd.DataFrame(first_stock_df.close.rolling(
        n_days_stride).corr(second_stock_df.close)).rename(
        {'close': f"{first_stock_name}_{second_stock_name}_close_corr_rolling_{n_days_stride}_days"},axis=1).fillna(method='backfill').round(6)

    
    # add cols
    
    current_feature_cols = list(final_stock_df.columns)
    

    # as we go through different pairs will have multiple var / corr for the first stock
    # pnc_bar calcualtes corr for pnr
    #pnr_bat calculates corr for pnr
    # don't want the same cols
    if f"{first_stock_name}_close_std_rolling_{n_days_stride}_days" not in current_feature_cols:
        
        rolling_close_std_first_stock =  first_stock_df.close.rolling(n_days_stride).std().fillna(method='backfill').round(6)
        rolling_close_df[f"{first_stock_name}_close_std_rolling_{n_days_stride}_days"] = rolling_close_std_first_stock
        
    if f"{second_stock_name}_close_std_rolling_{n_days_stride}_days" not in current_feature_cols:
        rolling_close_std_second_stock =  second_stock_df.close.rolling(n_days_stride).std().fillna(method='backfill').round( 6)
        rolling_close_df[f"{second_stock_name}_close_std_rolling_{n_days_stride}_days"] = rolling_close_std_second_stock
        
    if f"{first_stock_name}_volume_std_rolling_{n_days_stride}_days" not in current_feature_cols:
        rolling_volume_std_first_stock =  first_stock_df.volume.rolling(n_days_stride).std().fillna(method='backfill').round(6)
        rolling_close_df[f"{first_stock_name}_volume_std_rolling_{n_days_stride}_days"] = rolling_volume_std_first_stock
        
    if f"{second_stock_name}_volume_std_rolling_{n_days_stride}_days" not in current_feature_cols:
        rolling_volume_std_second_stock =  second_stock_df.volume.rolling(n_days_stride).std().fillna(method='backfill').round(6)
        rolling_close_df[f"{second_stock_name}_volume_std_rolling_{n_days_stride}_days"] = rolling_volume_std_second_stock
    
    return rolling_close_df



In [603]:
# 2 minutes fo 210 pairs
final_stock_df, pairs_of_stocks = build_correlation_dfs(dict_of_stocks_and_dfs, verbose=False)


Finished 0.0 pct of stocks


Finished 0.02564102564102564 pct of stocks


Finished 0.05128205128205128 pct of stocks


Finished 0.07692307692307693 pct of stocks


Finished 0.10256410256410256 pct of stocks


Finished 0.1282051282051282 pct of stocks


Finished 0.15384615384615385 pct of stocks


Finished 0.1794871794871795 pct of stocks


Finished 0.20512820512820512 pct of stocks


Finished 0.23076923076923078 pct of stocks


Finished 0.2564102564102564 pct of stocks


Finished 0.28205128205128205 pct of stocks


Finished 0.3076923076923077 pct of stocks


Finished 0.3333333333333333 pct of stocks


Finished 0.358974358974359 pct of stocks


Finished 0.38461538461538464 pct of stocks


Finished 0.41025641025641024 pct of stocks


Finished 0.4358974358974359 pct of stocks


Finished 0.46153846153846156 pct of stocks


Finished 0.48717948717948717 pct of stocks


Finished 0.5128205128205128 pct of stocks


Finished 0.5384615384615384 pct of stocks


Finished 0.5641025641025641 pct of 

In [ ]:
# max number of stocks is ~300 NOT 990

In [604]:
len(final_stock_df.columns)

4957

In [585]:
len(final_stock_df.columns)

5777

# Prep code for NN

In [1128]:
# prepare the data for LSTM model
def split_sequences(sequences, n_steps, y_col='pg_so_close_corr_rolling_7_days', start_idx=0, n_val=50, print_idx=100, input_verbose=1): #2200
    """
    sequences = input_data
    n_steps = n_days of data to give at a time
    
    only works for the currently set y_col
    """
    if y_col not in sequences.columns:
        raise ValueError('This y col does not exist in this df')
    
    X, y = list(), list()
    X_val, y_val = list(), list()
    
    n_sequences = len(sequences)
    print('n_sequences', n_sequences)

    for i in range(start_idx, n_sequences):
        if i == start_idx and input_verbose == 1:
            print(f"Training idx start at {i}")
        if (i % print_idx == 0) and (i != 0) and input_verbose==1:
            print(f"Pct finished = {i/n_sequences}")
            
        # find the end of this pattern
        end_ix = i + n_steps 
        total_end_ix = end_ix + n_val
        # check if we are beyond the dataset
        if (total_end_ix) > n_sequences:
            print(f"Training idx end at {end_ix}")
            print('Total idx checked', total_end_ix)
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = np.array(sequences.loc[:, sequences.columns != f"{y_col}"][i:end_ix]), np.array(
            sequences.loc[:, sequences.columns == f"{y_col}"].shift(-7).fillna(method='ffill').iloc[end_ix-1])

                                 
        X.append(seq_x)
        y.append(seq_y)
    
    val_start_idx = start_idx + n_sequences - (start_idx  + n_val -2)
    for i in range(val_start_idx, n_sequences):
        if i == val_start_idx and input_verbose==1:
            print(f"Val idx start at {val_start_idx}")
        if (i % print_idx == 0) and i != 0 and input_verbose==1:
            print(f"Pct finished for val sequences = {i/n_sequences}")
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences) and input_verbose==1:
            print(f"Val idx end at {end_ix}")
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = np.array(sequences.loc[:, sequences.columns != f"{y_col}"][i:end_ix]), np.array(
            sequences.loc[:, sequences.columns == f"{y_col}"].shift(-7).fillna(method='ffill').iloc[end_ix-1])
        
        
        X_val.append(seq_x)
        y_val.append(seq_y)
    
    

    X, y, X_val, y_val = array(X), array(y), array(X_val), array(y_val)
    
    # errors for standard scaler
    X = np.nan_to_num(X.astype(np.float32)) # converting to float 32 throws some infinity errors
    X_val = np.nan_to_num(X_val.astype(np.float32)) # converting to float 32 throws some infinity errors  
    
    
    scalers = {}
    for i in range(X.shape[1]):
        scalers[i] = StandardScaler()
        X[:, i, :] = scalers[i].fit_transform(X[:, i, :]) 
    
    pca_scalers = {}
    N_COMPONENTS=100

    new_X = np.zeros((X.shape[0], X.shape[1], N_COMPONENTS))
    for i in range(X.shape[1]):
        pca_scalers[i] = PCA(n_components=N_COMPONENTS) # ~80%
        new_X[:, i, :] = pca_scalers[i].fit_transform(X[:, i, :]) 


    for i in range(X_val.shape[1]):
        X_val[:, i, :] = scalers[i].transform(X_val[:, i, :]) 


    new_X_val = np.zeros((X_val.shape[0], X_val.shape[1], N_COMPONENTS))
    for i in range(X_val.shape[1]):
        new_X_val[:, i, :] = pca_scalers[i].transform(X_val[:, i, :]) 
        
   # need  to do this again as standard scaler may have nans
    X = np.nan_to_num(X.astype(np.float32)) # converting to float 32 throws some infinity errors
    X_val = np.nan_to_num(X_val.astype(np.float32)) # converting to float 32 throws some infinity errors 
    print('X val shape', X_val.shape)
    

    
    return new_X, y, new_X_val, y_val, scalers, pca_scalers

    
    

In [1141]:
X,y, X_val, y_val, scalers, pca_scalers = split_sequences(
    training_data,
    30, start_idx=0, input_verbose=1,
    n_val=40, y_col=f"wal_mtb_close_corr_rolling_7_days"
) # 30 steps

n_sequences 597
Training idx start at 0
Pct finished = 0.16750418760469013
Pct finished = 0.33500837520938026
Pct finished = 0.5025125628140703
Pct finished = 0.6700167504187605
Pct finished = 0.8375209380234506
Training idx end at 558
Total idx checked 598
Val idx start at 559
Val idx end at 598
X val shape (9, 30, 4956)


In [1106]:
def build_keras_model(n_steps, n_features, n_units=100, dropout_pct=0.05, n_layers = 1):
    model = Sequential()


    # define CNN model
#     model.add(TimeDistributed(Conv2D(n_units, kernel_and_pool_size))
#     model.add(TimeDistributed(MaxPooling2D(pool_size=kernel_and_pool_size))
#     model.add(TimeDistributed(Flatten()))

        
    model.add(LSTM(n_units, activation='relu', dropout=dropout_pct, return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(BatchNormalization())
    for _ in range(n_layers):
        model.add(LSTM(n_units, activation='relu', dropout=dropout_pct, return_sequences=True))
        model.add(BatchNormalization())
    model.add(LSTM(n_units, activation='relu', dropout=dropout_pct))
    model.add(BatchNormalization())
    model.add(Dense(n_units))
    model.add(Dense(int(n_units/2)))
    model.add(Dense(1))
    #Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, amsgrad=False)
    #LR = 0.0001
    #clipnorm=1., clipvalue=0.5
    model.compile(optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mse', metrics=['mse'])
    return model

In [1107]:
model = build_keras_model(30, 7610)

In [1108]:
model.summary()

Model: "sequential_94"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_264 (LSTM)              (None, 30, 100)           3084400   
_________________________________________________________________
batch_normalization_255 (Bat (None, 30, 100)           400       
_________________________________________________________________
lstm_265 (LSTM)              (None, 30, 100)           80400     
_________________________________________________________________
batch_normalization_256 (Bat (None, 30, 100)           400       
_________________________________________________________________
lstm_266 (LSTM)              (None, 100)               80400     
_________________________________________________________________
batch_normalization_257 (Bat (None, 100)               400       
_________________________________________________________________
dense_261 (Dense)            (None, 100)             

In [1109]:


class PredictionCallback(Callback):
    def on_epoch_end(self, epoch, logs={}):
        y_pred = self.model.predict(self.validation_data[0])
        print('prediction: {} at epoch: {}'.format(y_pred, epoch))


# Train a model

In [1137]:
# train on all data
# predict for the upcoming week

def prediction_for_upcoming_week(final_stock_df,pairs_of_stocks,  job_id=None, print_idx=1, n_day_sequences=14, 
                                 start_date_training_data='2018-01-01', n_validation_sequences=40, input_batch_size=128, 
                                 input_verbose=1):
    """
    The main entrypoint for training an LSTM network on stock predictions
    
    :param final_stock_df: The list of stock pairs with correlations over different time ranges, volume
    :para pairs_of_stocks: The list of stock pairs
    :param print_idx: The number of iterations to pass before printing out progress
    :param n_day_sequences: The number of sequences to pass to the LSTM (i.e. the number of days)
    :param start_date_training_data: Filter for data before thie date to train on
    :param n_validation_sequences: Number of sequences to validate on. Should be >= 40
    :param input_batch-size
    """
    # validation needs to be 40 or index error
    final_stock_df = final_stock_df.dropna()
    final_stock_df = final_stock_df.sort_values(by='date')
    # add this to predictions
    stock_to_industry = pd.read_csv('../data/Industries stock list - all.csv')
    stock_to_industry.symbol = [i.lower() for i in stock_to_industry.symbol]

    final_stock_df = final_stock_df.dropna()
    most_recent_date = final_stock_df.index.max()

    prediction_end = most_recent_date + datetime.timedelta(7)



    test_df = final_stock_df.iloc[-n_day_sequences:, :]

    

    n_days_corr_predictions = 7


    pct_change_corr = []
    predicted_corr = []
    last_corr_for_prediction_day = []
    pred_dates = []
    first_stock_industries = []
    second_stock_industries = []
    
    first_model = True

    start = time.time()
    total_n = len(pairs_of_stocks)
    
    for idx,stock_pairing in enumerate(pairs_of_stocks):
        if idx % print_idx == 0 and input_verbose ==1 :
            print('----------')
            print(f"Stock pairing = {stock_pairing}")
            print(f"Pct finished = {idx/total_n}")
        first_stock_name, second_stock_name = stock_pairing.split('_')

        first_stock_industries.append(stock_to_industry[stock_to_industry.symbol == first_stock_name].industry.values[0])
        second_stock_industries.append(stock_to_industry[stock_to_industry.symbol == second_stock_name].industry.values[0])



        pred_col_name = f"{stock_pairing}_close_corr_rolling_{n_days_corr_predictions}_days"

        # remove the current 7-day corr for this stock
        # for 7 take rolling 7 days corr to the present day to predict off of
        
        ## TRAINING AND TESTING DATA
        return final_stock_df[final_stock_df.index >= f"{start_date_training_data}"]
        X,y, X_val, y_val, scalers, pca_scalers = split_sequences(
            final_stock_df[final_stock_df.index >= f"{start_date_training_data}"],
            n_day_sequences, start_idx=0, input_verbose=input_verbose,
            n_val=n_validation_sequences, y_col=f"{pred_col_name}"
        ) # 30 steps
        return X,y, X_val, y_val, scalers, pca_scalers 

        

        train_X, train_y = final_stock_df.loc[:, final_stock_df.columns != f"{pred_col_name}"],  final_stock_df[f"{pred_col_name}"].shift(-7).fillna(method='ffill') 
                                                           # get corr from 7 days in the future
        test_X, test_y = np.array(test_df.loc[:, test_df.columns != f"{pred_col_name}"]),  test_df[f"{pred_col_name}"]
        test_X = test_X.reshape(1, test_X.shape[0], test_X.shape[1])
        test_X = np.nan_to_num(test_X.astype(np.float32))
        
        for i in range(test_X.shape[1]):
            test_X[:, i, :] = scalers[i].transform(test_X[:, i, :]) 
        test_X = np.nan_to_num(test_X.astype(np.float32))
        
        new_X_test = np.zeros((test_X.shape[0], test_X.shape[1], N_COMPONENTS))
        for i in range(test_X.shape[1]):
            new_X_test[:, i, :] = pca_scalers[i].transform(test_X[:, i, :]) 
        

        
#         return X,y, X_val, y_val, new_X_test
        ## END TRAINING AND TESTING DATA 
        
        
        if first_model:
            smaller_model = build_keras_model(X.shape[1],X.shape[2])
            print(smaller_model.summary())
            
        # test again at 700 epochs
        if first_model:
            start = time.time()
            # 800 epochs
#             early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=200, verbose=1, restore_best_weights=True)
            history = smaller_model.fit(x=X, y=y, batch_size=input_batch_size, epochs=200, verbose=input_verbose, 
                      validation_data=(X_val, y_val), shuffle=False,  use_multiprocessing=False, callbacks=[early_stopping])
            end=time.time()

            print((end-start)/60,' minutes')
        else:

            # Freeze the layers except the last 5 layers
            for layer in smaller_model.layers[:-3]:
                layer.trainable = False
            # Check the trainable status of the individual layers

#             for layer in smaller_model.layers:
#                 print(layer, layer.trainable)

#             smaller_model.compile(optimizer=Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False), loss='mse', metrics=['mse'])
            
            start = time.time()
#             early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1, restore_best_weights=True)
            smaller_model = build_keras_model(X.shape[1],X.shape[2])
            print(smaller_model.summary())
            history = smaller_model.fit(x=X, y=y, batch_size=input_batch_size, epochs=200, verbose=input_verbose, 
                      validation_data=(X_val, y_val), shuffle=False,  use_multiprocessing=False, callbacks=[early_stopping])
            end=time.time()
            print((end-start)/60,' minutes')

    
        history_df  = pd.DataFrame(history.history)
        history_df[['mse', 'val_mse']].iloc[-100:, :].plot()
        plt.show()
        prediction = smaller_model.predict(new_X_test)[0][0] 

        if idx % print_idx==0:
            print(f"Prediction = {prediction}")



        last_corr_date = train_y.index.max()
        last_corr = train_y[train_y.index.max()]  
        if idx % print_idx==0:
            print(f"Last corr = {last_corr}")

        pred_dates.append(most_recent_date)
        predicted_corr.append(prediction)
        last_corr_for_prediction_day.append(last_corr)
        
        if input_verbose==1 and idx % print_idx==0:
            print(f"{stock_pairing} corr7-day corr of close from {most_recent_date} to {prediction_end} is {prediction} ")
        
        first_model = False


    end = time.time()

    print(f"Predictions took {(end-start)/60} mins")

    squarred_difference = (np.array(last_corr_for_prediction_day)-np.array(predicted_corr))**2

    prediction_df = pd.DataFrame({ 'pred_date_start':pred_dates,'stock_pair':pairs_of_stocks,   'first_stock_industry': first_stock_industries, 
                   'second_stock_industry': second_stock_industries,
                   'predicted_corr': predicted_corr, 'last_7_day_corr_for_pred_date_start': last_corr_for_prediction_day, 
            'squarred_diff_7_day_cor': (np.array(last_corr_for_prediction_day)-np.array(predicted_corr))**2
                 })
    
    if job_id:
        tmp_filepath = '../data/lstm_tmp_prediction_dfs'
        if not os.path.isdir(f"{tmp_filepath}"):
            os.mkdir(f"{tmp_filepath}")
        prediction_df.to_csv(
        f'{tmp_filepath}/{job_id}_lstm_test_predictions_{most_recent_date}-{prediction_end}.csv', index=False)
    else:
        prediction_df.to_csv(
    f'../data/predictions/lstm_test_predictions_{most_recent_date}-{prediction_end}.csv', index=False)


In [1130]:
len(pairs_of_stocks)

741

In [1131]:
def chunks(l, n):
    return [l[i:i+n] for i in range(0, len(l), n)]

def do_job(job_id, stock_pairs, data_slice):
    all_prediction_df = []
    prediction_for_upcoming_week(data_slice, stock_pairs, job_id=job_id)



def dispatch_jobs(data, job_number, pairs):
    total = len(pairs)
    chunk_size = total / job_number
    slice = chunks(pairs, int(chunk_size))
    jobs = []

    for i, pair in enumerate(slice):
        j = Process(target=do_job, args=(i, pair, data))
        jobs.append(j)
    for j in jobs:
        j.start()

In [1113]:
if __name__ == '__main__':
    final_stock_df = final_stock_df.dropna()
    num_workers = mp.cpu_count()  
    dispatch_jobs(final_stock_df, num_workers , pairs_of_stocks[:100])

2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
2523 len input final_stock_df
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
n_sequences 597
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598
Training idx end at 568
Total idx checked 598


Process Process-21:
Process Process-19:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/jonathanhilgart/.pyenv/versions/3.7.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jonathanhilgart/.pyenv/versions/3.7.0/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/jonathanhilgart/.pyenv/versions/3.7.0/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-1112-bbbb3f9032f4>", line 6, in do_job
    prediction_for_upcoming_week(data_slice, stock_pairs, job_id=job_id)
  File "<ipython-input-1110-093a2f63fce5>", line 61, in prediction_for_upcoming_week
    n_val=n_validation_sequences, y_col=f"{pred_col_name}"
  File "<ipython-input-1105-c88a42d08761>", line 54, in split_sequences
    sequences.loc[:, sequences.columns == f"{y_col}"].shift(-7).fillna(method='ffill').iloc[end_ix])
  File "/U

In [1117]:
final_stock_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2523 entries, 2005-06-30 to 2020-05-15
Columns: 4957 entries, wal_mtb_close_corr_rolling_5_days to year
dtypes: float64(4953), int64(4)
memory usage: 95.4 MB


In [1138]:
# 6.5 minutes for 10 stocks
final_stock_df = final_stock_df.dropna()
# test 14 day period instead of 30
# no dice
# test smaller network smae LR
# no dice
# test since 2016 data
# no dice
# test smaller learning rate 
# no dice
# test smaller batch size
# nothing
# test batch size  


# so the solution was less validation data

# wal_cwbc is turns out needed less training data

# test a new model for each pair, 200 epochs per . 2 minutes per 200 epochs
# 300 pairs take 10 hours sequentially
# X,y, X_val, y_val, scalers, pca_scalers 
training_data= prediction_for_upcoming_week(final_stock_df, pairs_of_stocks[:200])

----------
Stock pairing = wal_mtb
Pct finished = 0.0


In [1135]:
training_data

,wal_mtb_close_corr_rolling_5_days,wal_close_std_rolling_5_days,mtb_close_std_rolling_5_days,wal_volume_std_rolling_5_days,mtb_volume_std_rolling_5_days,wal_volume,mtb_volume,wal_mtb_close_corr_rolling_7_days,wal_close_std_rolling_7_days,mtb_close_std_rolling_7_days,...,cpt_reg_close_corr_rolling_5_days,cpt_reg_close_corr_rolling_7_days,cpt_reg_close_corr_rolling_10_days,cpt_reg_close_corr_rolling_30_days,cpt_reg_close_corr_rolling_180_days,cpt_reg_close_corr_rolling_365_days,day,month,quarter,year
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,0.969491,0.242590,0.540629,71459.939653,82750.470242,389396.0,424392.0,0.982199,0.644249,0.961794,...,0.522325,0.887355,0.928270,0.098938,0.464441,-0.388416,2,1,1,2018
2018-01-03,0.666366,0.223271,0.870747,76557.999255,87630.342220,338118.0,437559.0,0.726948,0.438623,0.848135,...,-0.091648,0.765359,0.908782,0.126810,0.486648,-0.386528,3,1,1,2018
2018-01-04,0.721908,0.240520,1.102034,158775.994394,143127.496064,611753.0,628314.0,0.662799,0.207961,0.931213,...,0.962028,0.961260,0.953959,0.322918,0.508217,-0.386158,4,1,1,2018
2018-01-05,0.578692,0.463271,1.137299,128230.249714,94217.107971,425203.0,549691.0,0.590420,0.388716,0.969475,...,0.975050,0.977253,0.965353,0.471573,0.530037,-0.384616,5,1,1,2018
2018-01-08,0.280348,0.400849,0.870994,111338.714266,84419.407121,348035.0,488878.0,0.551488,0.407472,0.929618,...,0.981058,0.978178,0.967308,0.543734,0.548609,-0.381319,8,1,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-11,0.766228,0.868637,2.179638,52817.185241,149191.991991,757758.0,897767.0,0.748908,0.777775,2.790674,...,0.820796,0.621151,0.773188,0.790651,0.947226,0.762557,11,5,2,2020
2020-05-12,0.959172,1.505380,3.481016,93169.966213,232839.250455,842800.0,1054813.0,0.921566,1.291135,3.298373,...,0.881267,0.833468,0.809434,0.793117,0.947982,0.765509,12,5,2,2020
2020-05-13,0.982959,2.297035,5.954967,252559.966011,235833.161623,1243691.0,970433.0,0.965713,1.897660,5.166668,...,0.891453,0.860975,0.796259,0.795974,0.948399,0.768252,13,5,2,2020


In [ ]:
np.isnan(X_val).sum()

In [1021]:
np.isnan(new_X_test).sum()

0